In [ ]:
# Step 0: Install required libraries
!pip install torch sentencepiece pandas --quiet

In [ ]:
# Step 1: Generate and save Hindi dataset (80 rows x 5 columns)
import csv

data = [
    ("भारत ने ऑस्ट्रेलिया को पाँच विकेट से हराया।", "खेल", "सकारात्मक", 1),
    ("रोहित शर्मा ने शतक लगाते हुए टीम को जीत दिलाई।", "खेल", "सकारात्मक", 1),
    ("भारतीय हॉकी टीम ने स्वर्ण पदक जीता।", "खेल", "सकारात्मक", 1),
    ("पीवी सिंधु ने बैडमिंटन में रजत पदक हासिल किया।", "खेल", "सकारात्मक", 1),
    ("क्रिकेट विश्व कप में भारत सेमीफाइनल में पहुँचा।", "खेल", "सकारात्मक", 1),
    ("नीरज चोपड़ा ने भाला फेंक में नया रिकॉर्ड बनाया।", "खेल", "सकारात्मक", 1),
    ("भारत को फुटबॉल में करारी हार का सामना करना पड़ा।", "खेल", "नकारात्मक", 0),
    ("टेनिस टूर्नामेंट में भारतीय खिलाड़ी पहले दौर में बाहर हुए।", "खेल", "नकारात्मक", 0),
    ("कबड्डी विश्व कप में भारत उपविजेता रहा।", "खेल", "तटस्थ", 2),
    ("मैच बारिश के कारण रद्द कर दिया गया।", "खेल", "तटस्थ", 2),
    ("सरकार ने नई शिक्षा नीति लागू करने की घोषणा की।", "राजनीति", "सकारात्मक", 1),
    ("संसद में नया बजट पेश किया गया।", "राजनीति", "तटस्थ", 2),
    ("चुनाव आयोग ने मतदान की तारीख की घोषणा की।", "राजनीति", "तटस्थ", 2),
    ("प्रधानमंत्री ने विदेश यात्रा के दौरान कई समझौते किए।", "राजनीति", "सकारात्मक", 1),
    ("विपक्ष ने सरकार की नीतियों की आलोचना की।", "राजनीति", "नकारात्मक", 0),
    ("राज्य सरकार ने किसानों के लिए राहत पैकेज दिया।", "राजनीति", "सकारात्मक", 1),
    ("देश में महंगाई दर में वृद्धि दर्ज की गई।", "राजनीति", "नकारात्मक", 0),
    ("नए कानून को संसद ने मंजूरी दी।", "राजनीति", "तटस्थ", 2),
    ("मुख्यमंत्री ने जनता से सीधे संवाद किया।", "राजनीति", "सकारात्मक", 1),
    ("राजनीतिक दलों के बीच गठबंधन की चर्चा शुरू हुई।", "राजनीति", "तटस्थ", 2),
    ("सरकारी स्कूलों में डिजिटल क्लासरूम स्थापित किए जाएंगे।", "शिक्षा", "सकारात्मक", 1),
    ("बोर्ड परीक्षा के परिणाम घोषित हुए, छात्रों ने उत्कृष्ट प्रदर्शन किया।", "शिक्षा", "सकारात्मक", 1),
    ("IIT प्रवेश परीक्षा में इस वर्ष रिकॉर्ड आवेदन आए।", "शिक्षा", "तटस्थ", 2),
    ("विश्वविद्यालय में नए पाठ्यक्रम शुरू किए गए।", "शिक्षा", "सकारात्मक", 1),
    ("ग्रामीण क्षेत्रों में साक्षरता दर बढ़ी है।", "शिक्षा", "सकारात्मक", 1),
    ("परीक्षा में नकल के मामले सामने आए।", "शिक्षा", "नकारात्मक", 0),
    ("छात्रवृत्ति योजना से हजारों छात्रों को लाभ मिला।", "शिक्षा", "सकारात्मक", 1),
    ("शिक्षक भर्ती परीक्षा का परिणाम जारी किया गया।", "शिक्षा", "तटस्थ", 2),
    ("ऑनलाइन शिक्षा में तकनीकी खामियाँ सामने आईं।", "शिक्षा", "नकारात्मक", 0),
    ("सरकार ने मिड-डे मील योजना का विस्तार किया।", "शिक्षा", "सकारात्मक", 1),
    ("सरकार ने मुफ्त टीकाकरण अभियान शुरू किया।", "स्वास्थ्य", "सकारात्मक", 1),
    ("नई दवा के परीक्षण में सफलता मिली।", "स्वास्थ्य", "सकारात्मक", 1),
    ("देश में डेंगू के मामले तेजी से बढ़ रहे हैं।", "स्वास्थ्य", "नकारात्मक", 0),
    ("आयुष्मान भारत योजना से लाखों मरीजों को फायदा हुआ।", "स्वास्थ्य", "सकारात्मक", 1),
    ("अस्पताल में बेड की कमी के कारण मरीजों को परेशानी हुई।", "स्वास्थ्य", "नकारात्मक", 0),
    ("योग और आयुर्वेद की वैश्विक मांग बढ़ी है।", "स्वास्थ्य", "सकारात्मक", 1),
    ("नई स्वास्थ्य नीति का मसौदा तैयार किया गया।", "स्वास्थ्य", "तटस्थ", 2),
    ("कैंसर के इलाज में नई तकनीक ने उम्मीद जगाई।", "स्वास्थ्य", "सकारात्मक", 1),
    ("वायु प्रदूषण के कारण सांस की बीमारियाँ बढ़ी हैं।", "स्वास्थ्य", "नकारात्मक", 0),
    ("स्वास्थ्य विभाग ने जागरूकता अभियान चलाया।", "स्वास्थ्य", "सकारात्मक", 1),
    ("भारत ने स्वदेशी 5G तकनीक विकसित की।", "प्रौद्योगिकी", "सकारात्मक", 1),
    ("ISRO ने सफलतापूर्वक उपग्रह लॉन्च किया।", "प्रौद्योगिकी", "सकारात्मक", 1),
    ("साइबर हमलों में वृद्धि देखी जा रही है।", "प्रौद्योगिकी", "नकारात्मक", 0),
    ("आर्टिफिशियल इंटेलिजेंस से रोजगार के नए अवसर आए।", "प्रौद्योगिकी", "सकारात्मक", 1),
    ("डेटा प्राइवेसी बिल संसद में पेश किया गया।", "प्रौद्योगिकी", "तटस्थ", 2),
    ("स्टार्टअप इंडिया ने नए रिकॉर्ड निवेश आकर्षित किए।", "प्रौद्योगिकी", "सकारात्मक", 1),
    ("मोबाइल इंटरनेट की गति में सुधार आया है।", "प्रौद्योगिकी", "सकारात्मक", 1),
    ("सोशल मीडिया पर फर्जी खबरों का प्रसार चिंताजनक।", "प्रौद्योगिकी", "नकारात्मक", 0),
    ("भारतीय IT कंपनियों ने वैश्विक बाजार में धूम मचाई।", "प्रौद्योगिकी", "सकारात्मक", 1),
    ("ड्रोन तकनीक का कृषि में उपयोग बढ़ा।", "प्रौद्योगिकी", "सकारात्मक", 1),
    ("बॉलीवुड फिल्म ने पहले हफ्ते में 200 करोड़ की कमाई की।", "मनोरंजन", "सकारात्मक", 1),
    ("OTT प्लेटफॉर्म पर हिंदी कंटेंट की मांग बढ़ी।", "मनोरंजन", "सकारात्मक", 1),
    ("फिल्म समारोह में भारतीय फिल्म को पुरस्कार मिला।", "मनोरंजन", "सकारात्मक", 1),
    ("प्रसिद्ध अभिनेता ने सामाजिक कार्य में भाग लिया।", "मनोरंजन", "सकारात्मक", 1),
    ("संगीत एल्बम ने करोड़ों स्ट्रीम का आँकड़ा पार किया।", "मनोरंजन", "सकारात्मक", 1),
    ("फिल्म की शूटिंग के दौरान हादसा हुआ।", "मनोरंजन", "नकारात्मक", 0),
    ("टेलीविजन रेटिंग में नाटकीय बदलाव देखा गया।", "मनोरंजन", "तटस्थ", 2),
    ("कॉमेडी शो ने दर्शकों में खूब लोकप्रियता पाई।", "मनोरंजन", "सकारात्मक", 1),
    ("पुरानी फिल्मों को रीमेक किए जाने की लहर चली।", "मनोरंजन", "तटस्थ", 2),
    ("नई वेब सीरीज को दर्शकों ने खूब सराहा।", "मनोरंजन", "सकारात्मक", 1),
    ("शेयर बाजार ने नई ऊँचाई छुई।", "व्यापार", "सकारात्मक", 1),
    ("भारत की GDP वृद्धि दर 7% रही।", "व्यापार", "सकारात्मक", 1),
    ("नए FDI नियमों से विदेशी निवेश बढ़ा।", "व्यापार", "सकारात्मक", 1),
    ("कच्चे तेल की कीमतों में उछाल से महंगाई बढ़ी।", "व्यापार", "नकारात्मक", 0),
    ("छोटे और मध्यम उद्योगों को सरकारी सहायता मिली।", "व्यापार", "सकारात्मक", 1),
    ("रुपए की कीमत में गिरावट आई।", "व्यापार", "नकारात्मक", 0),
    ("भारतीय कंपनी ने विदेशी बाजार में कदम रखा।", "व्यापार", "सकारात्मक", 1),
    ("व्यापार घाटे में मामूली कमी दर्ज की गई।", "व्यापार", "तटस्थ", 2),
    ("स्टार्टअप इकोसिस्टम में नए यूनिकॉर्न उभरे।", "व्यापार", "सकारात्मक", 1),
    ("बाजार में मंदी के संकेत दिखे।", "व्यापार", "नकारात्मक", 0),
    ("भारतीय वैज्ञानिकों ने नई प्रजाति की खोज की।", "विज्ञान", "सकारात्मक", 1),
    ("चंद्रयान मिशन ने अहम डेटा भेजा।", "विज्ञान", "सकारात्मक", 1),
    ("जलवायु परिवर्तन पर नई रिपोर्ट जारी हुई।", "विज्ञान", "तटस्थ", 2),
    ("अंतरिक्ष अनुसंधान में नया मील का पत्थर हासिल हुआ।", "विज्ञान", "सकारात्मक", 1),
    ("ग्लेशियर पिघलने की दर चिंताजनक स्तर पर पहुँची।", "विज्ञान", "नकारात्मक", 0),
    ("नई ऊर्जा स्रोत की खोज में सफलता मिली।", "विज्ञान", "सकारात्मक", 1),
    ("भूकंप की भविष्यवाणी के लिए नई तकनीक विकसित।", "विज्ञान", "सकारात्मक", 1),
    ("समुद्री प्रदूषण पर शोध रिपोर्ट सामने आई।", "विज्ञान", "नकारात्मक", 0),
    ("भारतीय शोधकर्ताओं का शोध अंतरराष्ट्रीय जर्नल में प्रकाशित।", "विज्ञान", "सकारात्मक", 1),
    ("सौरमंडल के बारे में नई जानकारी मिली।", "विज्ञान", "सकारात्मक", 1),
]

with open("hindi_dataset.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "text", "category", "sentiment", "label"])
    for i, (text, category, sentiment, label) in enumerate(data, 1):
        writer.writerow([i, text, category, sentiment, label])

print(f"Dataset saved: hindi_dataset.csv  ({len(data)} rows x 5 columns)")

In [ ]:
# Step 2: Load and explore the dataset
import pandas as pd

df = pd.read_csv("hindi_dataset.csv")
print(f"Shape: {df.shape}")
display(df.head(10))
print("\nCategory distribution:")
print(df["category"].value_counts())
print("\nSentiment distribution:")
print(df["sentiment"].value_counts())

In [ ]:
# Step 3: Build raw Hindi text corpus from all sentences
raw_text = " ".join(df["text"].tolist())
print(f"Total characters : {len(raw_text):,}")
print(f"Total sentences  : {len(df)}")
print(f"\nSample:\n{raw_text[:300]}")

In [ ]:
# Step 4: Hindi tokenization using Unicode Devanagari regex
import re

def hindi_tokenize(text):
    return re.findall(r'[\u0900-\u097F]+|[A-Za-z0-9]+|[।,!?]', text)

sample = df["text"][0]
tokens = hindi_tokenize(sample)
print(f"Original : {sample}")
print(f"Tokens   : {tokens}")
print(f"Count    : {len(tokens)}")

In [ ]:
# Step 5: Build vocabulary with special tokens <PAD> <UNK> <BOS> <EOS>
all_tokens = []
for text in df["text"]:
    all_tokens.extend(hindi_tokenize(text))

vocab = {"<PAD>": 0, "<UNK>": 1, "<BOS>": 2, "<EOS>": 3}
for idx, token in enumerate(sorted(set(all_tokens)), start=4):
    vocab[token] = idx
id_to_token = {v: k for k, v in vocab.items()}

print(f"Vocabulary size : {len(vocab)}")
print(f"Total tokens    : {len(all_tokens)}")
print("\nSample vocab:")
for token, idx in list(vocab.items())[4:14]:
    print(f"  '{token}' -> {idx}")

In [ ]:
# Step 6: Encode and decode Hindi sentences using the vocabulary
def encode(text, vocab):
    return [vocab.get(t, vocab["<UNK>"]) for t in hindi_tokenize(text)]

def decode(ids, id_to_token):
    return " ".join([id_to_token.get(i, "<UNK>") for i in ids])

sample_text = df["text"][5]
encoded = encode(sample_text, vocab)
print(f"Original : {sample_text}")
print(f"Encoded  : {encoded}")
print(f"Decoded  : {decode(encoded, id_to_token)}")

In [ ]:
# Step 7: Train SentencePiece BPE tokenizer on the Hindi corpus
import sentencepiece as spm

with open("hindi_corpus.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(df["text"].tolist()))

spm.SentencePieceTrainer.train(
    input="hindi_corpus.txt",
    model_prefix="hindi_bpe",
    vocab_size=500,
    character_coverage=1.0,
    model_type="bpe",
    pad_id=0, unk_id=1, bos_id=2, eos_id=3
)
sp = spm.SentencePieceProcessor()
sp.load("hindi_bpe.model")
print(f"BPE tokenizer trained | vocab size: {sp.get_piece_size()}")

In [ ]:
# Step 8: Encode Hindi text with BPE and show subword pieces
sample = df["text"][0]
pieces = sp.encode(sample, out_type=str)
ids    = sp.encode(sample, out_type=int)

print(f"Original  : {sample}")
print(f"BPE pieces: {pieces}")
print(f"BPE IDs   : {ids}")
print(f"Decoded   : {sp.decode(ids)}")

In [ ]:
# Step 9: Build padded token ID tensor for all 80 sentences
import torch

MAX_LEN = 20

def encode_and_pad(text, sp, max_len):
    ids = sp.encode(text, out_type=int)[:max_len]
    ids += [0] * (max_len - len(ids))
    return ids

token_tensor = torch.tensor(
    [encode_and_pad(t, sp, MAX_LEN) for t in df["text"]], dtype=torch.long
)
print(f"Token tensor shape : {token_tensor.shape}  -> (80 sentences x 20 tokens)")
print(f"Sample row         : {token_tensor[0].tolist()}")

In [ ]:
# Step 10: Token embedding layer for Hindi BPE vocabulary
VOCAB_SIZE = sp.get_piece_size()
EMB_DIM    = 64

torch.manual_seed(42)
token_embedding_layer = torch.nn.Embedding(VOCAB_SIZE, EMB_DIM, padding_idx=0)
token_embeddings = token_embedding_layer(token_tensor)
print(f"Token embeddings shape : {token_embeddings.shape}  -> (80 x 20 x 64)")

In [ ]:
# Step 11: Positional embedding layer for sequence positions
pos_embedding_layer = torch.nn.Embedding(MAX_LEN, EMB_DIM)
positions = torch.arange(MAX_LEN).unsqueeze(0).expand(len(df), -1)
pos_embeddings = pos_embedding_layer(positions)
print(f"Positional embeddings shape : {pos_embeddings.shape}")

In [ ]:
# Step 12: Final input embeddings = token embeddings + positional embeddings
input_embeddings = token_embeddings + pos_embeddings
print(f"Token embeddings        : {token_embeddings.shape}")
print(f"Positional embeddings   : {pos_embeddings.shape}")
print(f"Final input embeddings  : {input_embeddings.shape}")

In [ ]:
# Step 13: Mean pooling to get one sentence-level vector per sentence
sentence_vectors = input_embeddings.mean(dim=1)
print(f"Sentence vectors shape : {sentence_vectors.shape}  -> (80 x 64)")
print(f"\nVector sample (first 8 dims) : {sentence_vectors[0][:8].tolist()}")
print(f"Sentence                     : {df['text'][0]}")

In [ ]:
# Step 14: Cosine similarity — same vs different category sentences
import torch.nn.functional as F

v1 = sentence_vectors[0]    # खेल
v2 = sentence_vectors[1]    # खेल
v3 = sentence_vectors[20]   # शिक्षा

sim_same = F.cosine_similarity(v1.unsqueeze(0), v2.unsqueeze(0)).item()
sim_diff = F.cosine_similarity(v1.unsqueeze(0), v3.unsqueeze(0)).item()

print(f"S1 : {df['text'][0]}")
print(f"S2 : {df['text'][1]}")
print(f"S3 : {df['text'][20]}")
print(f"\nSimilarity S1 vs S2 (same - खेल)    : {sim_same:.4f}")
print(f"Similarity S1 vs S3 (diff - शिक्षा) : {sim_diff:.4f}")

In [ ]:
# Step 15: Full Hindi Word Embedding pipeline summary
print("=" * 55)
print("    HINDI WORD EMBEDDING PIPELINE SUMMARY")
print("=" * 55)
print(f"Dataset          : hindi_dataset.csv")
print(f"Shape            : {df.shape[0]} rows x {df.shape[1]} columns")
print(f"Categories       : {sorted(df['category'].unique())}")
print(f"Corpus size      : {len(raw_text):,} characters")
print(f"Tokenizer        : SentencePiece BPE (Hindi-trained)")
print(f"BPE Vocab size   : {VOCAB_SIZE}")
print(f"Embedding dim    : {EMB_DIM}")
print(f"Sequence length  : {MAX_LEN}")
print(f"Token emb shape  : {token_embeddings.shape}")
print(f"Final emb shape  : {input_embeddings.shape}")
print(f"Sentence vectors : {sentence_vectors.shape}")
print("=" * 55)
print("Hindi embeddings ready!")